# Fold network and load data in the neurons


While there are whole organism based datasets are increasingly becoming common for C elegans, several datasets are not currently available for every individual worm neuron, but a subset. Even for whole organism level datasets, data can often be found organized by neuron classes.
For example, datasets can be found with neuron class based on left/right position (AWCL and AWCR into AWC), or by dorsal and ventral position (RMEDL/RMEVL), or by lineage/function (e.g. amphids or ventral cord neurons (VC)) or by some combination of these. Here we show by taking the example of the CENGEN dataset how the network can be folded across these axes, preserving the data and combining the connections across the axis of folding.

In [ ]:
### How are the neurotransmitter release ligands defined?

In [ ]:
from cedne import utils
import pandas as pd
import matplotlib
import matplotlib.patches as mpatches
import numpy as np

## Specify worm parameters

In [ ]:
w = utils.makeWorm()

## Optional parameters
w.stage='L4'
w.sex='Hermaphrodite'
w.genotype='N2'

## Neural network for this worm
nn = w.networks['Neutral']

## Loading CENGEN data into a dataframe (Using threshold 4 data)

In [ ]:
transcripts = pd.read_csv(utils.thres_4,encoding= 'unicode_escape', index_col=1).drop(['Wormbase_ID','Unnamed: 0'], axis = 'columns')

## Creating the folding dictionary

The folding dictionary has the class name as the key and a list of neuron names that belong to that class as the values.
All the neurons in the values of this dictionary will be folded together into a single "class neuron" with its connections and 
attributes being the union of the connections and attributes of its member neurons.
Folding dictionaries can also be created by loading from a csv file. 

**Note that AWC_OFF has been mapped to AWCL and AWC_ON to AWCR for the graph.**

In [ ]:
suffixes = ['', 'D', 'V', 'L', 'R', 'DL', 'DR', 'VL', 'VR', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13']
neuron_class = {}
for n in transcripts.columns:
    neuron_class[n] = []
    for s in suffixes:
        if (n+s) in nn.neurons:
            neuron_class[n].append(n+s)
    if n == 'AWC_OFF':
        neuron_class[n].append('AWCL')
    if n == 'AWC_ON':
        neuron_class[n].append('AWCR')
    if n == 'VD_DD':
        for m in nn.neurons:
            if m.startswith('VD') or m.startswith('DD'):
                neuron_class[n].append(m)
    if n == 'RME_LR':
        for m in nn.neurons:
            if m == 'RMEL' or m == 'RMER':
                neuron_class[n].append(m)
    if n == 'RME_DV':
        for m in nn.neurons:
            if m == 'RMED' or m == 'RMEV':
                neuron_class[n].append(m)
    if n == 'RMD_LR':
        for m in nn.neurons:
            if m == 'RMDL' or m == 'RMDR':
                neuron_class[n].append(m)
    if n == 'RMD_DV':
        for m in nn.neurons:
            if m.startswith('RMDD') or m.startswith('RMDV'):
                neuron_class[n].append(m)
    if n == 'IL2_LR':
        for m in nn.neurons:
            if m == 'IL2L' or m == 'IL2R':
                neuron_class[n].append(m)
    if n == 'IL2_DV':
        for m in nn.neurons:
            if m.startswith('IL2D') or m.startswith('IL2V'):
                neuron_class[n].append(m)

for p in ['DA09']:
    neuron_class['DA'].remove(p)
for p in ['VB01', 'VB02']:
    neuron_class['VB'].remove(p)
for p in ['VC04', 'VC05']:
    neuron_class['VC'].remove(p)
for p in ['DB01']:
    neuron_class['DB'].remove(p) 
for p in ['VA12']:
    neuron_class['VA'].remove(p)

neuron_class['VC_4_5'] = ['VC04', 'VC05']
neuron_class['DA9'] = ['DA09']

In [ ]:
neuron_class

In [ ]:
nn.nodes

## Folding the network

In [ ]:
nn_folded = nn.fold_network(neuron_class)

## Loading cengen transcripts into the network

In [ ]:
for n in nn_folded.neurons:
    nn_folded.neurons[n].set_property('transcript', transcripts[n])

In [ ]:
## Giving the sex specific neurons an interneuron type for positioning on graph.
sex_neurons = ['CAN']
for n in nn_folded.neurons:
    if n in sex_neurons:
        nn_folded.neurons[n].type = 'interneuron'

## Plotting the network with the CENGEN transcript data

In [ ]:
nn.edges

In [ ]:
transcript_name = 'gpa-4'
save = False #outputDir + transcript_name + '.pdf'
conns = [(e[0].name, e[1].name) for e in nn_folded.edges]
connNodes = list(nn_folded.neurons.keys())
nodeColors = {n:('orange' if nn_folded.neurons[n].transcript[transcript_name]>0 else 'lightgray') for n in connNodes}

pos = utils.plot_layered(conns, neunet=nn_folded, nodeColors=nodeColors, title= '', save= save)

ASI looks interesting...

## Plotting the subnetwork of the neuron 'ASI'

In [ ]:
conns = nn_folded.neurons['ASI'].get_connections()
g = nn_folded.subnetwork(connections=conns, as_view=False)
asi_fig = utils.plot_shell(g, center='ASI', figsize=(6,6))

In [ ]:
### Put the synapse number on the edges, gap junction. edge weight as the number of synapses.

## Fold by category in the network.

In [ ]:
utils.loadNeurotransmitters(nn)

In [ ]:
by_category = {}
for n in nn.neurons:
    if not nn.neurons[n].category in by_category: 
        by_category[nn.neurons[n].category] = []
    by_category[nn.neurons[n].category].append(n)

In [ ]:
nn_cat = nn.fold_network(by_category, data='clean')

In [ ]:
edges = list(nn_cat.connections.keys())
for e in edges:
    c = nn_cat.connections[e]
    if c.connection_type != 'chemical-synapse':
        nn_cat.connections.pop(e)

In [ ]:
fig_ent = utils.plot_shell(nn_cat, save='chemical-synapses-grouped.svg', shells=[[], [nn_cat.neurons[n] for n in nn_cat.neurons]], figsize=(8,8), width_logbase=10)

In [ ]:
edge_nt = {}
for e,c in nn_cat.connections.items():
    for cid, conn in c.contraction_data.items():
        if(conn.connection_type) == 'chemical-synapse':
            if len(conn.putative_neurotrasmitter_receptors):
                if all([isinstance(ntrec,str) for ntrec in set(list(zip(*conn.putative_neurotrasmitter_receptors))[0])]):
                    edge_nt[e] = '_'.join(set(list(zip(*conn.putative_neurotrasmitter_receptors))[0]))
                else:
                    edge_nt[e] = 'others'
            else:
                    edge_nt[e] = 'others'
        else:
                    edge_nt[e] = 'others'

In [ ]:
for e,c in nn_cat.connections.items():
    print(e,c.connection_type)
    for cid, conn in c.contraction_data.items():
        pass

In [ ]:
setedge_nt = set(edge_nt.values())
setedge_nt.remove('others')
set_edge_nt = list(setedge_nt) + ['others']
edge_cm = list(matplotlib.cm.plasma(np.linspace(0,1,len(set_edge_nt)-1))) + [np.array(matplotlib.colors.to_rgba('gray'))]
edge_color_dict = {eid:edge_cm[set_edge_nt.index(ent)] for eid, ent in edge_nt.items()}
edge_alpha_dict = {eid:1 for eid, ent in edge_nt.items()}
handles = [mpatches.Patch(color=color, label=label) for color, label in zip(edge_cm, set_edge_nt)]

In [ ]:
fig_ent = utils.plot_shell(nn_cat, shells=[[], [nn_cat.neurons[n] for n in nn_cat.neurons]], figsize=(12,12), width_logbase=10, edge_color_dict=edge_color_dict, edge_alpha_dict=edge_alpha_dict, handles=handles)

In [ ]:
set_edge_nt[:-1]

In [ ]:
edge_color_dict

In [ ]:
handle_dict = {ent:edge_cm[set_edge_nt.index(ent)] for eid, ent in edge_nt.items() if nt in ent}
handle_dict['others'] = edge_cm[-1]
handles = [mpatches.Patch(color=color, label=label) for label,color in handle_dict.items()]
print(handles)

In [ ]:
for nt in set_edge_nt[:-1]:
    edge_color_dict = {eid:edge_cm[set_edge_nt.index(ent)] if nt in ent else edge_cm[-1] for eid, ent in edge_nt.items()}
    edge_alpha_dict = {eid:1 if nt in ent else 0.1 for eid, ent in edge_nt.items()}
    handle_dict = {ent:edge_cm[set_edge_nt.index(ent)] for eid, ent in edge_nt.items() if nt in ent}
    handle_dict['others'] = edge_cm[-1] 
    handles = [mpatches.Patch(color=color, label=label) for label,color in handle_dict.items()]
    # handles = [mpatches.Patch(color=color, label=label) for color, label in zip([edge_cm[set_edge_nt.index(nt)], edge_cm[-1]], [nt, 'others'])]
    fig_ent = utils.plot_shell(nn_cat, shells=[[], [nn_cat.neurons[n] for n in nn_cat.neurons]], figsize=(8,8), width_logbase=10, edge_color_dict=edge_color_dict, edge_alpha_dict=edge_alpha_dict, handles=handles, save=f"grouped-{nt}_new.svg")